## Set up notebook

In [1]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
%matplotlib inline

!{sys.executable} -m pip install opencv-python

import time
import scipy
import numpy as np
from PIL import Image
from scipy import ndimage
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

np.random.seed(123)

/Users/viviandang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/viviandang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/viviandang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/viviandang/opt/anaconda3/lib/python3.7/site-pack

Using TensorFlow backend.


## Reading data

In [2]:
# import cv2

In [3]:
# DATADIR = 'data/Train/'
# CATEGORIES = ['NORMAL', 'PNEUMONIA']
# for category in CATEGORIES:
#     path = os.path.join(DATADIR, category)
#     for img in os.listdir(path):
#         img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
#         plt.imshow(img_array,)
#         plt.show()
#         break
#     break

## Create path to folders

In [4]:
import os, shutil

## Scale and Reshape

In [5]:
# import time
# import scipy
# import numpy as np
# from PIL import Image
# from scipy import ndimage
# from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

# np.random.seed(123)

In [6]:
test_folder = 'data/test'
train_folder = 'data/train'
val_folder = 'data/val'

In [7]:
# get all the data in the directory split/test (180 images), and reshape them
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_folder, 
        target_size=(64, 64), batch_size = 624 ) 

# get all the data in the directory split/validation (200 images), and reshape them
val_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(val_folder, 
        target_size=(64, 64), batch_size = 16)

# get all the data in the directory split/train (542 images), and reshape them
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_folder, 
        target_size=(64, 64), batch_size = 5216)

Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 5216 images belonging to 2 classes.


In [8]:
train_images, train_labels = next(train_generator)
test_images, test_labels = next(test_generator)
val_images, val_labels = next(val_generator)

In [9]:
print(train_labels)

[[1. 0.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [10]:
# Explore your dataset again
m_train = train_images.shape[0]
num_px = train_images.shape[1]
m_test = test_images.shape[0]
m_val = val_images.shape[0]

print ("Number of training samples: " + str(m_train))
print ("Number of testing samples: " + str(m_test))
print ("Number of validation samples: " + str(m_val))
print ("train_images shape: " + str(train_images.shape))
print ("train_labels shape: " + str(train_labels.shape))
print ("test_images shape: " + str(test_images.shape))
print ("test_labels shape: " + str(test_labels.shape))
print ("val_images shape: " + str(val_images.shape))
print ("val_labels shape: " + str(val_labels.shape))

Number of training samples: 5216
Number of testing samples: 624
Number of validation samples: 16
train_images shape: (5216, 64, 64, 3)
train_labels shape: (5216, 2)
test_images shape: (624, 64, 64, 3)
test_labels shape: (624, 2)
val_images shape: (16, 64, 64, 3)
val_labels shape: (16, 2)


In [11]:
train_img = train_images.reshape(train_images.shape[0], -1)
test_img = test_images.reshape(test_images.shape[0], -1)
val_img = val_images.reshape(val_images.shape[0], -1)

print(train_img.shape)
print(test_img.shape)
print(val_img.shape)

(5216, 12288)
(624, 12288)
(16, 12288)


In [12]:
train_y = np.reshape(train_labels[:,0], (5216,1))
test_y = np.reshape(test_labels[:,0], (624,1))
val_y = np.reshape(val_labels[:,0], (16,1))

## Creating X-y

In [13]:
from keras.preprocessing.image import ImageDataGenerator

## Build a baseline model (fully connected)

In [14]:
# Build a baseline fully connected model
from keras import models
from keras import layers
np.random.seed(123)
model = models.Sequential()
model.add(layers.Dense(20, activation='relu', input_shape=(12288,))) # 2 hidden layers
model.add(layers.Dense(7, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [15]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [16]:
# pip install keras

In [26]:
from tensorflow import keras
from keras import models
from keras import layers
import keras_metrics

ModuleNotFoundError: No module named 'keras_metrics'

In [23]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['acc',f1_m,precision_m, recall_m])

histoire = model.fit(train_img,
                    train_y,
                    epochs=50,
                    batch_size=16,
                    validation_data=(val_img, val_y), verbose = 1)

# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(test_img, test_y, verbose=1)

Train on 5216 samples, validate on 16 samples
Epoch 1/50
5216/5216 [==============================] - 2s 327us/step - loss: 0.1403 - acc: 0.9567 - f1_m: 0.8847 - precision_m: 0.9164 - recall_m: 0.8769 - val_loss: 0.4874 - val_acc: 0.6875 - val_f1_m: 0.5455 - val_precision_m: 1.0000 - val_recall_m: 0.3750
Epoch 2/50
5216/5216 [==============================] - 1s 172us/step - loss: 0.1371 - acc: 0.9597 - f1_m: 0.8972 - precision_m: 0.9382 - recall_m: 0.8804 - val_loss: 0.3598 - val_acc: 0.9375 - val_f1_m: 0.9333 - val_precision_m: 1.0000 - val_recall_m: 0.8750
Epoch 3/50
5216/5216 [==============================] - 1s 166us/step - loss: 0.1458 - acc: 0.9576 - f1_m: 0.8952 - precision_m: 0.9367 - recall_m: 0.8780 - val_loss: 0.2350 - val_acc: 0.8750 - val_f1_m: 0.8750 - val_precision_m: 0.8750 - val_recall_m: 0.8750
Epoch 4/50
5216/5216 [==============================] - 1s 165us/step - loss: 0.1384 - acc: 0.9578 - f1_m: 0.8918 - precision_m: 0.9378 - recall_m: 0.8755 - val_loss: 0.4537 

In [21]:
loss

0.41661950945854187

In [24]:
print(loss, accuracy, f1_score, precision, recall)

0.9509993592898051 0.7419871687889099 0.48266419768333435 0.9624999761581421 0.33071663975715637


In [ ]:
results_train = model.evaluate(train_img, train_y)

In [ ]:
results_train

In [ ]:
results_val = model.evaluate(val_img, val_y)

In [ ]:
results_val

In [ ]:
model.summary()